# Training OVERLAECOBO
En este notebook se llevaran acabo las pruebas con OVERLAECOBO, se probaran los distintos ZOOMS y la informacion extra para asi escoger la mejor configuracion de dataset y de red neuronal. Como primer paso sera obtener el repositorio donde estan las redes entrenadas y preparalo para su funcionamiento con OVERLAECOBO

In [ ]:
!git clone https://github.com/GeorgeSeif/Semantic-Segmentation-Suite.git

¿Que pruebas se van a realizar?
* Un entrenamiento con cada uno de los zooms sin informacion aumentada
* Un entrenamiento con cada uno de los zooms con informacion aumentada

Las pruebas tendran la etiqueta del lugar donde fueron realizadas, esto debido a que hoy (24 - 05 -18) aun no tengo una cuenta e guane, sin olvidar que cuando la tenga debo hacer configuraciones e instalar tensorflow lo que conllevara bastante tiempo, tiempo en el que podria hacer pruebas en mi pc. Tambien se documentará los parametros a usar, epoch, batch_size, etc.

## 1. Preparar el dataset para usar la Suite
Dado que la suite tiene su propia estructura para el dataset, se hace necesario cambiar los directorios a dicha estructura, y aqui se presenta un inconveniente y es que segun un [issue](https://github.com/GeorgeSeif/Semantic-Segmentation-Suite/issues/65) que encontre hoy (24 - 05 - 18 7:18 am) y segun eso de hace una hora, en el framework no usan las imagenes de test, entonces:
1.  mi test se vuelve el nuevo valid y no existitian imagenes para test y siguo usando el framework
2. cambio todo lo interno del repo para que use test y duro otra semana sacando mi dataset con la nueva division de 3 partes y no solo dos como lo hice
3. busco otro repositorio

Me quedaré con la segunda opcion, la tercera no me cuadra ya con este esta sirviendo, podria atacar el problema con otras cuestiones, primero el issue es muy reciente y podria estar pendiente de cuando haga los arreglos y pues solo seria volver a copiar el repo y si no dan solucion tan solo crear un metodo para sacar un score, para predecir se predice con una imagen, le paso for a la carpeta test, que predicga y luego trato de sacar algun score haciedno comparaciones alfin y alcabo la cuestion es de que entrene

### 1.1 Restructurando dataset
La suite pide una estrucutra de dataset de la siguiente manera.

           - dataset
               - train
               - train_labels
               - valid
               - valid_labels
               - class_dict.scv

cabe aclarar que esta vendria siendo la estructura para un unico zoom asi que dentro de la carpeta actual de OVERLAECOBO creare nuevas carpetas con cada zoom y luego dentro dividir en train y valid.

In [ ]:
import os

In [ ]:
for i in range(8):
    os.mkdir("./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13))

In [ ]:
for i in range(8):
    os.rename("./data/dataset/OVERLAECOBO/train/satellite/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/train")
    os.rename("./data/dataset/OVERLAECOBO/train/economic_layer/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/train_labels")
    os.rename("./data/dataset/OVERLAECOBO/test/satellite/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/val") 
    os.rename("./data/dataset/OVERLAECOBO/test/economic_layer/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/val_labels")    

Queria copiar con codigo los archivos class_dict pero al parece la libreia os no tiene un `cp` y con shutil habia como 4 metodos para copiar, aisque lo hice manual
### 1.2 Switch Train Test
Dado que al parecer las carpetas valid tienes mas datos de las de train hare un switch entoncs valid pasara a ser train y train valid


In [ ]:
for i in range(7):
    os.rename("./OVERLAECOBO_"+str(i+14)+"/train","./OVERLAECOBO_"+str(i+14)+"/pibote")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/train_labels","./OVERLAECOBO_"+str(i+14)+"/pibote_labels")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/valid","./OVERLAECOBO_"+str(i+14)+"/train")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/valid_labels","./OVERLAECOBO_"+str(i+14)+"/train_labels")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/pibote","./OVERLAECOBO_"+str(i+14)+"/val")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/pibote_labels","./OVERLAECOBO_"+str(i+14)+"/val_labels")   

## 2. Metricas
Al parecer el framework usa varias metricas provenientes de sklearn, y lo se dado a que en uno de sus archivos `.py` se encuentra el siguiente import

      from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
   
y mirando en la documentacion de sklearn, dichas metricas significan [precision_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score), [recall_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score), [f1_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score)

## 3. Pruebas
Y por ahora (24 - May - 2018) que aun no tengo la cuenta de guane iniciare las pruebas e mi GPU y con los ZOOMs mas grandes puesto que hay menos imagenes. una de las cosas es que podria probar varios modelos, entre los que permite la suite

* DenseNet56
* FC-DenseNet67
* FC-DenseNet103 
* Encoder-Decoder
* Encoder-Decoder-Skip
* RefineNet-Res50
* RefineNet-Res101 
* RefineNet-Res152
* FRRN-A
* FRRN-B
* MobileUNet
* MobileUNet-Skip
* PSPNet-Res50 
* PSPNet-Res101
* PSPNet-Res152
* GCN-Res50 
* GCN-Res101
* GCN-Res152

Para cada prueba se realizara un train, un test y un predcit. el predict será de una imagen cualquiera del directorio valid

### 3.1 Sin Información extra

Las pruebas a realizar a continuacion son pruebas echas meramente con los 3 canales de una imagen RGB, para las pruebas con informacion extra se agregaran uno o mas canales, dependiendo de la informacion obtenida y la capacidad del framework

## Prueba multi capa

In [6]:
import os
from skimage import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [ ]:
for i in range(7):
    img = io.imread("./OVERLAECOBO_13_multi/train/"+str(i)+".png")
    d = img.flatten()
    f = list(d) + list(d)
    g = np.array(f).reshape((256,256,6))[:,:,:4]
    io.imsave("./OVERLAECOBO_13_multi/train/"+str(i)+".png",g)

In [ ]:
img = io.imread("./OVERLAECOBO_13_multi/train/0.tiff")
img.shape

### 4. Pruebas
Cada una de las pruebas tendra su propio notebook por cada nivel de Zoom

<center><h1> RESULTADOS</h1></center>

Acontinuacion los resultados de las distintas pruebas realizadas.


In [8]:
PATH_PRUEBAS = "./Pruebas/OVERLAECOBO_#"

## OVERLAECOBO 13

In [4]:
ZOOM = "13"

Las pruebas realizadas con OVERLAECOBO_13 fueron

In [9]:
pruebas = pd.read_csv(PATH_PRUEBAS.replace("#","13")+"/pruebas.csv")
pruebas

,PRUEBA,RED,EPOCH,BATCH_SIZE
0,1,FC-DenseNet56,50,1
1,2,FC-DenseNet56,70,1
2,3,Encoder-Decoder,70,1
3,4,Encoder-Decoder,70,2
4,5,Encoder-Decoder-Skip,70,1
5,6,Encoder-Decoder-Skip,70,2
6,7,FRRN-A,70,1
7,8,MobileUNet,70,1
8,9,MobileUNet,70,2
9,10,MobileUNet-Skip,70,1


#### TRAIN RESULTS

In [45]:
S = PATH_PRUEBAS.replace("#",ZOOM)+"/prueba_"+str(pruebas["PRUEBA"][0])+"/train.txt"
S

'./Pruebas/OVERLAECOBO_13/prueba_1/train.txt'

El archivo train generado: [Train File](./Pruebas/OVERLAECOBO_13/prueba_1/train.txt)

In [42]:
S = PATH_PRUEBAS.replace("#",ZOOM)+"/prueba_"+str(pruebas["PRUEBA"][0])+"/accuracy_vs_epochs.png"
S

'./Pruebas/OVERLAECOBO_13/prueba_1/accuracy_vs_epochs.png'

![](./Pruebas/OVERLAECOBO_13/prueba_1/accuracy_vs_epochs.png)

In [43]:
S = PATH_PRUEBAS.replace("#",ZOOM)+"/prueba_"+str(pruebas["PRUEBA"][0])+"/loss_vs_epochs.png"
S

'./Pruebas/OVERLAECOBO_13/prueba_1/loss_vs_epochs.png'

![](./Pruebas/OVERLAECOBO_13/prueba_1/loss_vs_epochs.png)

#### TEST RESULTS

In [25]:
resultados_val =  pd.read_csv(PATH_PRUEBAS.replace("#",ZOOM)+"/prueba_"+str(pruebas["PRUEBA"][0])+"/Val/val_scores.csv")
resultados_val

,val_name,avg_accuracy,precision,recall,f1 score,mean iou,Estrato_1,Estrato_2,Estrato_3,Estrato_4,Estrato_5,Estrato_6,No_Definido
0,0,0.109436,0.403108,0.109436,0.078118,0.039587,0.082335,0.251546,0.080670,0.033435,1.0,1.0,0.611111
1,1,0.304077,0.444509,0.304077,0.318972,0.121578,0.104298,0.194425,0.095327,1.000000,1.0,1.0,0.704956


In [46]:
S = PATH_PRUEBAS.replace("#",ZOOM)+"/prueba_"+str(pruebas["PRUEBA"][0])+"/test.txt"
S

'./Pruebas/OVERLAECOBO_13/prueba_1/test.txt'

Archivo test generado: [Test File](./Pruebas/OVERLAECOBO_13/prueba_1/test.txt)

In [1]:
import shutil

In [18]:
ZOOMS = ["14_COMPOUND"]
for i in ZOOMS:
    for j in pruebas["PRUEBA"]:
        shutil.copyfile("./Pruebas/.gitignore", PATH_PRUEBAS.replace("#",i)+"/prueba_"+str(pruebas["PRUEBA"][j-1])+"/Val/.gitignore")